In [47]:
from natsort import natsorted

## sys
import random
import time
from glob import glob
import json
from pathlib import Path
import os

## numeric
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F

## vis
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.colors import LinearSegmentedColormap
from mpl_toolkits import mplot3d
from matplotlib import collections  as mc
from mpl_toolkits.mplot3d.art3d import Line3DCollection

## notebook
from IPython import display
from tqdm import tqdm_notebook as tqdm

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

plt.style.use('ggplot')
plt.style.use('seaborn-colorblind')



def dict2tensor(d, fill=None):
    n = len(d.keys())
    k2i = {k:i for i,k in enumerate(natsorted(d.keys()))}
    res = torch.zeros(len(d.keys()), len(d.keys()), device=device)
    for src_node, dst_nodes in d.items():
        for dst_node, distance in dst_nodes.items():
            if fill is not None:
                res[k2i[src_node],k2i[dst_node]] = fill
            else:
                res[k2i[src_node],k2i[dst_node]] = distance
    return res, k2i


def graph2json(graph, D, W, initPositions):
    res = {k:v for k,v in graph.items()}
    res['nodes'] = [
        {
            'index':i, 
            'id': node['id'],
            'x': initPositions[i]['x'],
            'y': initPositions[i]['y'],
        } 
        for i, node in enumerate(graph['nodes'])]
    res['weight'] = W.cpu().numpy().tolist()
    res['graphDistance'] = D.cpu().numpy().tolist()
    return res

## preprocess graph for contest

In [51]:
fns = natsorted(
#     glob('contest-prep-0/*.json')
#     + glob('contest-prep-0/*.txt')
    glob('contest-0/*.json')
    + glob('contest-0/*.txt')
)
fns

[]

In [48]:
for fn in fns:
    print(' in: ' + fn)
    with open(fn) as f:
        graph = json.load(f)
    graph['nodes'] = natsorted(graph['nodes'], key=lambda x:x['id'])
    
    nodes = [n['id'] for n in graph['nodes']]
    edges = [[e['source'], e['target']] for e in graph['edges']]
    
    G = nx.Graph()
    G.add_nodes_from(nodes)
    G.add_edges_from(edges)
    
    nodeCount = len(G.nodes)
    D, k2i = dict2tensor(dict(nx.all_pairs_shortest_path_length(G)))
    Adj,_ = dict2tensor(dict(G.adjacency()), fill=1)
    
    W = 1/(D**2+np.eye(nodeCount))
    js = graph2json(graph, D, W, graph['nodes'])
    
    fnout = fn.replace('-0', '-1').replace('txt', 'json')
    print(f'out: {fnout}')
    if not Path(Path(fnout).parent).exists():
        os.makedirs(Path(fnout).parent)
    with open(fnout, 'w') as f:
        json.dump(js, f, indent=2)
        print()
    print('-'*40)   

 in: contest-prep-0/test-1.json
out: contest-prep-1/test-1.json

----------------------------------------
 in: contest-prep-0/test-1.txt
out: contest-prep-1/test-1.json

----------------------------------------
 in: contest-prep-0/test-2.txt
out: contest-prep-1/test-2.json

----------------------------------------
 in: contest-prep-0/test-3.txt
out: contest-prep-1/test-3.json

----------------------------------------
 in: contest-prep-0/test-4.txt
out: contest-prep-1/test-4.json

----------------------------------------


In [39]:
for fn in fns:
    graph_name = Path(fn).name.split('.')[0]
    print(f"<option value='{graph_name}'>{graph_name}</option>")

<option value='test-1'>test-1</option>
<option value='test-1'>test-1</option>
<option value='test-2'>test-2</option>
<option value='test-3'>test-3</option>
<option value='test-4'>test-4</option>


<hr>

## generate a graph

In [4]:
# fns = natsorted(
#     glob('_tsne_output/*.json')
#     +glob('_neato_sfdp_layouts0/*.dot'),
#     key=lambda x:x.split('/')[-1]
# )
# fns = [fn for fn in fns if 'grid' in fn and 'tnse']







['contest-prep/test-1.json']

In [52]:

# for fn in fns:
#     print(' in: ' + fn)
#     with open(fn) as f:
#         graph = json.load(f)
#     print(graph['nodes'])
#     for n in graph['nodes']:
#         n['id'] = str(n['id'])
        
#     for e in graph['edges']:
#         e['source'] = str(e['source'])
#         e['target'] = str(e['target'])
        
#     for p in graph['initPositions']:
#         p['id'] = str(p['id'])
    
#     for d, pos in zip(graph['nodes'], graph['initPositions']):
#         assert d['id'] == pos['id'], f"{d['id']} != {pos['id']}"
#         d['x'] = pos['x']
#         d['y'] = pos['y']
    
#     graph['nodes'] = natsorted(graph['nodes'], key=lambda x:x['id'])
    
#     for n in graph['nodes']:
#         print(n)
#     nodes = [n['id'] for n in graph['nodes']]
    
#     print(nodes)
#     edges = [[e['source'], e['target']] for e in graph['edges']]
    
#     G = nx.Graph()
#     G.add_nodes_from(nodes)
#     G.add_edges_from(edges)
    
#     nodeCount = len(G.nodes)
#     D, k2i = dict2tensor(dict(nx.all_pairs_shortest_path_length(G)))
#     Adj,_ = dict2tensor(dict(G.adjacency()), fill=1)
    
#     eye = torch.eye(nodeCount, device=device)
#     W = 1/(D**2+eye)
#     print(len(G.nodes), len(graph['nodes']))
#     js = graph2json(G, D, W, graph['nodes'])
#     fnout = fn.split('/')[-1].replace('.dot', '.json')
#     if 'neato' in fnout:
#         fnout = 'neato_layouts/' + fnout
#     elif 'sfdp' in fnout:
#         fnout = 'sfdp_layouts/' + fnout
#     elif 'tsne' in fnout:
#         fnout = 'tsne_layouts/' + fnout
#     else:
#         raise Exception
        
#     with open(fnout, 'w') as f:
#         json.dump(js, f, indent=2)
#         print(f'out: {fnout}')
#         print()
        
        
#     if 'neato.dot' in fn:
#         init_neato = [[d['x'], d['y']] for d in graph['nodes']]
#     if 'sfdp.dot' in fn:
#         init_sfdp = [[d['x'], d['y']] for d in graph['nodes']]
# #     if 'tsne.json' in fn:
# #         init_tsne = [[d['x'], d['y']] for d in graph['nodes']]
        
#         js['initPosition_neato'] = init_neato
#         js['initPosition_sfdp'] = init_sfdp
# #         js['initPosition_tsne'] = init_tsne
        
#         fnout = '_'.join(fnout.split('/')[-1].split('_')[:-1]) + '.json'
#         with open(fnout, 'w') as f:
#             json.dump(js, f, indent=2)
#             print('out: ' + fnout)
#         print('-'*20)
        

In [53]:
# nx.draw_networkx(G, pos={str(i):pos for i,pos in enumerate(init_neato)})